<a href="https://colab.research.google.com/github/elibtronic/2025_state_of_scholarly_communication/blob/main/YEAR_Green_OA_Email_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Email template builder


In [ ]:
!pip install crossrefapi

from crossref.restful import Works
import pandas as pd
from zipfile import ZipFile, ZIP_DEFLATED
import glob
import os
import requests
import json
from google.colab import files
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

print("ready to proceed")

In [ ]:
# @title Green OA email template builder
# @markdown Not going to put this in the public repo as it has contact info
# @markdown Instead a Google Sheet shared as CSV
green_csv = "" # @param {"type":"string"}
from_mail = "" # @param {"type":"string"}
# @markdown The following is where the Google Sheet of author info is

# @markdown


green_df = pd.read_csv(green_csv)


In [ ]:
#view some entries if needed
green_df.tail(5)

In [ ]:
# Be sure to modify the message to personalize to your institution

def generate_email(doi,source, email,name,form_url):

  print("Working on... ",doi)
  try:
    l_name = str.strip(name.split(",")[0])
    g_name = str.strip(name.split(",")[1])
  except:
    l_name = "NA"
    g_name = "NA"

  try:
    email = email.strip()
  except:
    email = "NA@NA.com"

  works = Works()
  result = works.doi(doi)
  a_title = result['title'][0]

  html_content = (
  "<html>\n"
  "<head></head>\n"
  "<body>\n"
  f"<p>Hello {g_name},</p>\n"
  f"<p>Your article: <strong>{a_title}</strong>, "
  f"with DOI: <a href='{doi}'>{doi}</a> "
  "can be deposited in the Brock University Digital Repository.\n</br>"
  "What this means is that your paper will end up reaching a wider audience, which will increase engagement with your work and a possible increase in citations too.\n"
  "The library assessed the journal articles published by Brock authors in the past year and identified ones where a free version could be added to the Brock repository — including yours."
  " That is why you are receiving this email.</p>\n"
  f"<p>Not much is required from you, just the file, and this whole process shouldn't take much of your time to complete. This link: <a href='{form_url}'>{form_url}</a> will provide more information and start the deposit process if you are interested.</p>\n"
  "<p>More information about this program and why we are doing it can be found on our <a href='https://researchguides.library.brocku.ca/getting-your-research-out/green-open-access'>Getting Your Research Out</a> guide. I'm also happy to answer any questions you have.</p>"
  "<p>Thanks,</p>"
  "<p>  Tim</p>"
  "<p></p>"
  "<p>  Scholarly Publishing and Platforms Librarian</p>"
  "</body>\n"
  "</html>\n"
  )

  f_name = doi.split('https://doi.org/')[1].replace("/","_")

  # Create a MIMEText object for the HTML part
  html_part = MIMEText(html_content, 'html')

  # Create a MIMEMultipart object to hold the email parts
  msg = MIMEMultipart('alternative')
  msg['Subject'] = "About your paper in "+source
  msg['From'] = from_mail
  msg['To'] = email

  # Attach the HTML part to the message
  msg.attach(html_part)

  # The 'msg.as_string()' method will give you the full email message string
  # ready to be sent via an SMTP server.
  email_message_string = msg.as_string()

  #print(email_message_string)

  with open(f_name+".emltpl", 'w') as f:
    f.write(email_message_string)



In [ ]:
#delete any emails from last run just in case

for f_to_remove in glob.glob("*.emltpl"):
  os.remove(f_to_remove)

In [ ]:

#go through DF and make the emails
for green_article in green_df.itertuples():
  generate_email(green_article.DOI, \
               green_article.source,\
               green_article.email,\
               green_article.name,\
               green_article.form_url)

print("done")

In [ ]:
#Zip and Download email templates

zip_name = "green_machine_emails.zip"

with ZipFile(zip_name, "w", compression=ZIP_DEFLATED) as zf:
  for tef in glob.glob("*.emltpl"):
    zf.write(tef)

files.download(zip_name)